# Using NannyML Performance Estimation Algorithm from AWS Marketplace 

## Overview

NannyML can estimate the performance of a machine learning model running in production. You can read more about how it works [here](https://nannyml.readthedocs.io/en/stable/how_it_works/performance_estimation.html).


This sample notebook shows you how to use [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o) from AWS Marketplace to estimate the performance of your deployed machine learnign models.

Performance Estimation can work for binary classification, multiclass classification and regression.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o). 

## Contents
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure and visualize train and test dataset](#B.-Configure-and-visualize-train-and-test-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Train a machine learning model](#3:-Train-a-machine-learning-model)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Train a model](#3.2-Train-a-model)
1. [Deploy model](#4:-Deploy-model)
1. [Perform Batch inference](#6.-Perform-Batch-inference)
1. [Clean-up](#7.-Clean-up)
	1. [Delete the model](#A.-Delete-the-model)
	1. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))


## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o)
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

<font color='red'>Directly copy your assigned ARN code below:<font>

In [1]:
# algo_arn = "<Customer to specify algorithm ARN corresponding to their AWS region>"
with open('arn.txt', 'r') as file:
    algo_arn = file.read().rstrip()
# algo_arn

## 2. Prepare dataset

In [2]:
import sagemaker as sage
from sagemaker import get_execution_role
import pandas as pd
import json

### A. Dataset format expected by the algorithm

To fully demonstrate the capabilities of NannyML's performance estimation we will provide code for all 3 supported machine learning problem types.

- For Binary Classification we are going to use [NannyML's synthetic car loan dataset](https://nannyml.readthedocs.io/en/stable/datasets/binary_car_loan.html).
- For Multiclass Classification we are going to use [NannyML's synthetic multiclass creadit card assignment dataset](https://nannyml.readthedocs.io/en/stable/datasets/multiclass.html).
- For Regression we are going to use [NannyML's synthetic car price dataset](https://nannyml.readthedocs.io/en/stable/datasets/regression.html).


You can find some information about dataset format in **Usage Information** section of [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o).
<br>
More detailed information can be found in [NannyML's Data Requirements Documentation](https://nannyml.readthedocs.io/en/stable/tutorials/data_requirements.html).

<font color='red'>Edit code below as appropriate for the Machine Learning problem type you are interested in:<font>

In [3]:
machine_learning_problem_type = "Binary Classification"
# machine_learning_problem_type = "Multiclass Classification"
# machine_learning_problem_type = "Regression"

### B. Configure and visualize reference dataset

In [4]:
if machine_learning_problem_type == "Binary Classification":
    reference_dataset = "data/bc_reference.csv"
elif machine_learning_problem_type == "Multiclass Classification":
    reference_dataset = "data/mc_reference.csv"
elif machine_learning_problem_type == "Regression":
    reference_dataset = "data/reg_reference.csv"
else:
    raise ValueError("Unsupported Machine Learning Problem Type.")

# Show selected dataset
pd.read_csv(reference_dataset).head()

,car_value,salary_range,debt_to_income_ratio,loan_length,repaid_loan_on_prev_car,size_of_downpayment,driver_tenure,repaid,timestamp,y_pred_proba,y_pred
0,39811.0,40K - 60K €,0.632950,19.0,False,40%,0.212653,1.0,2018-01-01 00:00:00.000,0.99,1
1,12679.0,40K - 60K €,0.718627,7.0,True,10%,4.927549,0.0,2018-01-01 00:08:43.152,0.07,0
2,19847.0,40K - 60K €,0.721724,17.0,False,0%,0.520817,1.0,2018-01-01 00:17:26.304,1.00,1
3,22652.0,20K - 20K €,0.705992,16.0,False,10%,0.453649,1.0,2018-01-01 00:26:09.456,0.98,1
4,21268.0,60K+ €,0.671888,21.0,True,30%,5.695263,1.0,2018-01-01 00:34:52.608,0.99,1


### C. Upload datasets to Amazon S3

In [5]:
sagemaker_session = sage.Session()
bucket = sagemaker_session.default_bucket()
# bucket

In [6]:
demo_prefix = "doc-notebook-demo"

In [7]:
reference_data = sagemaker_session.upload_data(
    reference_dataset, bucket=bucket, key_prefix=demo_prefix
)

## 3: Train a machine learning model

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to train a machine learning model. 

### 3.1 Set up environment

In [8]:
role = get_execution_role()

In [9]:
output_location = f"s3://{bucket}/{demo_prefix}/output"
# output_location

### 3.2 Train a model

You can also find more information about dataset format in **Hyperparameters** section of [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o).

For even more detailed information you read NannyML tutorials on performance estimation for:
- [Binary Classification](https://nannyml.readthedocs.io/en/stable/tutorials/performance_estimation/binary_performance_estimation.html)
- [Multiclass Classification](https://nannyml.readthedocs.io/en/stable/tutorials/performance_estimation/multiclass_performance_estimation.html)
- [Regression](https://nannyml.readthedocs.io/en/stable/tutorials/performance_estimation/regression_performance_estimation.html)

In [16]:
# Define hyperparameters
if machine_learning_problem_type == "Binary Classification":
    nannyml_parameters = {
        "y_pred_proba": "y_pred_proba",
        "y_pred": "y_pred",
        "y_true": "repaid",
        "timestamp_column_name": "timestamp",
        "metrics": ["roc_auc"],
        "chunk_size": 5000,
        "problem_type": "classification_binary",
    }
    # json.dumps needed due to sagemaker specifications
    sagemaker_hyperparameters = {
        "data_filename": reference_dataset.split("/")[-1],
        "data_type": "csv",
        "problem_type": "classification_binary",
        "parameters": json.dumps(nannyml_parameters),
    }
elif machine_learning_problem_type == "Multiclass Classification":
    nannyml_parameters = {
        "y_pred": "y_pred",
        "y_pred_proba": {
            "prepaid_card": "y_pred_proba_prepaid_card",
            "highstreet_card": "y_pred_proba_highstreet_card",
            "upmarket_card": "y_pred_proba_upmarket_card"
        },
        "y_true": "y_true",
        "timestamp_column_name": "timestamp",
        "metrics": ["roc_auc"],
        "chunk_size": 5000,
        "problem_type": "classification_multiclass",
    }
    # json.dumps needed due to sagemaker specifications
    sagemaker_hyperparams = {
        "data_filename": "mc_reference.csv",
        "data_type": "csv",
        "problem_type": "classification_multiclass",
        "parameters": json.dumps(nannyml_parameters),
    }
elif machine_learning_problem_type == "Regression":
    nannyml_parameters = {
        "feature_column_names": [
            "car_age",
            "km_driven",
            "price_new",
            "accident_count",
            "door_count",
            "fuel",
            "transmission",
        ],
        "y_pred": "y_pred",
        "y_true": "y_true",
        "timestamp_column_name": "timestamp",
        "metrics": ["rmse"],
        "chunk_size": 6000,
        "tune_hyperparameters": False,
    }
    # json.dumps needed due to sagemaker specifications
    sagemaker_hyperparams = {
        "data_filename": reference_dataset.split("/")[-1],
        "data_type": "csv",
        "problem_type": "regression",
        "parameters": json.dumps(nannyml_parameters),
    }
else:
    raise ValueError("Unsupported Machine Learning Problem Type.")

For information on creating an `Estimator` object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [17]:
# Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name='nml-perf-est',
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    hyperparameters=sagemaker_hyperparameters,
)

In [18]:
# Run the training job.
estimator.fit(
    {'training': reference_data}
)

INFO:sagemaker:Creating training-job with name: nml-perf-est-2023-08-18-15-32-25-195


2023-08-18 15:32:25 Starting - Starting the training job...
2023-08-18 15:32:42 Starting - Preparing the instances for training......
2023-08-18 15:33:55 Downloading - Downloading input data
2023-08-18 15:33:55 Training - Downloading the training image......
2023-08-18 15:34:56 Uploading - Uploading generated training model.INFO:nannyml:Logger object created.
INFO:nannyml:Hyperparameters read.
INFO:nannyml:Estimator Instantiated.
INFO:nannyml:Loaded data.
INFO:nannyml.base:fitting nannyml.performance_estimation.confidence_based.cbpe.CBPE
DEBUG:nannyml.usage_logging:found NML_DISABLE_USAGE_LOGGING key in environment variables. Usage event CBPE estimator run not logged.
DEBUG:nannyml.usage_logging:found NML_DISABLE_USAGE_LOGGING key in environment variables. Usage event CBPE estimator fit not logged.
INFO:nannyml:Estimator fit.
INFO:nannyml.io.store.base:storing object "nannyml.performance_estimation.confidence_based.cbpe.CBPE" to store "nannyml.io.store.file_store.FilesystemStore"
DEBUG

See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/jobs/) and monitor the metrics/logs in **Monitor** section.

## 4: Deploy model

**NannyML's Performance Estimation is not designed for real time inference, therefore it is not recommended to use it in this way.**

For this reason we are not showcasing the real-time inference feature of Sagemaker Algorithm.

## 5. Perform Batch inference

In this section, you will perform batch inference using multiple input payloads together.

In [20]:
# upload the batch-transform job input files to S3

if machine_learning_problem_type == "Binary Classification":
    inference_dataset = "data/bc_analysis.csv"
elif machine_learning_problem_type == "Multiclass Classification":
    inference_dataset = "data/mc_analysis.csv"
elif machine_learning_problem_type == "Regression":
    inference_dataset = "data/reg_analysis.csv"
else:
    raise ValueError("Unsupported Machine Learning Problem Type.")

inference_data = sagemaker_session.upload_data(inference_dataset, bucket=bucket, key_prefix=demo_prefix)
# print("Transform input uploaded to " + inference_data)

In [22]:
# Run the batch-transform job
transformer = estimator.transformer(
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=output_location
)
transformer.transform(inference_data, content_type="text/csv")
transformer.wait()

INFO:sagemaker:Creating model package with name: nannyml-performance-estimation-1350cd6b-2023-08-18-15-51-26-673


.........

INFO:sagemaker:Creating model with name: nannyml-performance-estimation-1350cd6b-2023-08-18-15-52-12-259


INFO:sagemaker:Creating transform job with name: nml-perf-est-2023-08-18-15-52-15-502


.......................... * Serving Flask app '/web_app_serve.py'
 * Debug mode: off
#033#033WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.#033
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://127.0.0.1:8080
#033Press CTRL+C to quit#033
169.254.255.130 - - [18/Aug/2023 15:56:42] "GET /ping HTTP/1.1" 200 -
169.254.255.130 - - [18/Aug/2023 15:56:42] "#033GET /execution-parameters HTTP/1.1#033" 404 -
Received POST invocation request.
Estimation invoked with 50000 rows
Estimation invoked with columns: ['car_value', 'salary_range', 'debt_to_income_ratio', 'loan_length', 'repaid_loan_on_prev_car', 'size_of_downpayment', 'driver_tenure', 'timestamp', 'y_pred_proba', 'y_pred']
169.254.255.130 - - [18/Aug/2023 15:56:43] "POST /invocations HTTP/1.1" 200 -

2023-08-18T15:56:42.090:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
 * Serving

In [30]:
# output is available on following path
transformer.output_path#.split("/")#[-1]

's3://sagemaker-eu-central-1-684294718906/nml-perf-est-2023-08-18-15-52-15-502'

**View Results of Performance Estimation**

In [36]:
results = pd.read_csv(transformer.output_path + "/" + inference_dataset.split("/")[-1] + ".out", header = [0,1])
results

chunk                                                          
              key chunk_index start_index end_index           start_date   
0        [0:4999]           0           0      4999  2018-01-01 00:00:00  \
1     [5000:9999]           1        5000      9999  2018-01-31 06:36:00   
2   [10000:14999]           2       10000     14999  2018-03-02 13:12:00   
3   [15000:19999]           3       15000     19999  2018-04-01 19:48:00   
4   [20000:24999]           4       20000     24999  2018-05-02 02:24:00   
5   [25000:29999]           5       25000     29999  2018-06-01 09:00:00   
6   [30000:34999]           6       30000     34999  2018-07-01 15:36:00   
7   [35000:39999]           7       35000     39999  2018-07-31 22:12:00   
8   [40000:44999]           8       40000     44999  2018-08-31 04:48:00   
9   [45000:49999]           9       45000     49999  2018-09-30 11:24:00   
10       [0:4999]           0           0      4999  2018-10-30 18:00:00   
11    [5000:9999]           1        5000      9999  2018-11-30 00:36:00   
12  [10000:14999]           2       10000     14999  2018-12-30 07:12:00   
13  [15000:19999]           3       15000     19999  2019-01-29 13:48:00   
14  [20000:24999]           4       20000     24999  2019-02-28 20:24:00   
15  [25000:29999]           5       25000     29999  2019-03-31 03:00:00   
16  [30000:34999]           6       30000     34999  2019-04-30 09:36:00   
17  [35000:39999]           7       35000     39999  2019-05-30 16:12:00   
18  [40000:44999]           8       40000     44999  2019-06-29 22:48:00   
19  [45000:49999]           9       45000     49999  2019-07-30 05:24:00   

                                         roc_auc                            
                   end_date     period     value sampling_error  realized   
0   2018-01-31 06:27:16.848  reference  0.971219       0.001811  0.976253  \
1   2018-03-02 13:03:16.848  reference  0.971052       0.001811  0.969045   
2   2018-04-01 19:39:16.848  reference  0.970985       0.001811  0.971742   
3   2018-05-02 02:15:16.848  reference  0.971196       0.001811  0.971642   
4   2018-06-01 08:51:16.848  reference  0.971054       0.001811  0.969085   
5   2018-07-01 15:27:16.848  reference  0.971065       0.001811  0.967364   
6   2018-07-31 22:03:16.848  reference  0.970880       0.001811  0.968692   
7   2018-08-31 04:39:16.848  reference  0.971173       0.001811  0.970205   
8   2018-09-30 11:15:16.848  reference  0.971319       0.001811  0.974096   
9   2018-10-30 17:51:16.848  reference  0.971140       0.001811  0.971757   
10  2018-11-30 00:27:16.848   analysis  0.970755       0.001811       NaN   
11  2018-12-30 07:03:16.848   analysis  0.971022       0.001811       NaN   
12  2019-01-29 13:39:16.848   analysis  0.971418       0.001811       NaN   
13  2019-02-28 20:15:16.848   analysis  0.971103       0.001811       NaN   
14  2019-03-31 02:51:16.848   analysis  0.971135       0.001811       NaN   
15  2019-04-30 09:27:16.848   analysis  0.961105       0.001811       NaN   
16  2019-05-30 16:03:16.848   analysis  0.961839       0.001811       NaN   
17  2019-06-29 22:39:16.848   analysis  0.961088       0.001811       NaN   
18  2019-07-30 05:15:16.848   analysis  0.962548       0.001811       NaN   
19  2019-08-29 11:51:16.848   analysis  0.961331       0.001811       NaN   

                                                                         
   upper_confidence_boundary lower_confidence_boundary upper_threshold   
0                   0.976652                  0.965787         0.97866  \
1                   0.976484                  0.965620         0.97866   
2                   0.976417                  0.965553         0.97866   
3                   0.976628                  0.965764         0.97866   
4                   0.976487                  0.965622         0.97866   
5                   0.976497                  0.965633         0.97866   
6                   0.976312                  0.9654

## 7. Clean-up

### A. Delete the model

In [38]:
transformer.delete_model()

INFO:sagemaker:Deleting model with name: nannyml-performance-estimation-1350cd6b-2023-08-18-15-52-12-259


### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

